# Okta + GitGuardian
## Scripts for dealing with incidents when the developer is no longer active 

While we all would love to think our teammates will be around forever, or at least for the life of the company. The truth is sometimes life has other plans. 

A common scenario for GitGuardian customers, especially when doing historical scans, is that the developer who originally committed the code is long gone. 

This presents two major problems for an organization:
1 - The need to know how many incidents are affected and get a report of each one.
2 - Since the dev is gone, the incidents most likely need to be assigned to someone who has deeper familiarity with the projects involved.  

The following scripts are examples that can help solve these issues.  

This is written to be run manually and makes certain assumptions about your manager/owner access to both Okta and GitGuardian. 

This is all in the experimental stages but has been tested and works on my machine. 


## Okta 1 - Build List Of Affected Incidents


- Looks at the most recent triggered or assigned incidents 
    - the upper limit is 100 per page (default 10 for testing)
- Looks up a list of all the non-active emails in Okta
- Compare the assigned user's email to the Okta list
    - if the assigned user is not active, unassign the incident

Before we can work with Okta, we must set two variables.  Since we never want to hard code any credentials here, I am suggesting for testing reasons we use our .gitignored `authorization.py` file to set:
- okta_token = Our Okta API token created from your settings on their platform
- okta_subdomain = the workspace instance you are calling, for example dev-111111-admin from the URL https://dev-111111-admin.okta.com

Also note, Okta will not return lists of deactivated users, just suspended ones. This is why I used the approach of looking up the active users and working from there. 

In [ ]:
import requests
import authorization
import json
import time

auth_headers = {'Authorization' : 'Token {}'.format(authorization.token) }
okta_auth_header = {"Authorization": "SSWS {}".format(authorization.okta_token) }
okta_subdomain = authorization.okta_subdomain

# Get our current list of TRIGGERED incidents, listed with the most recent first.
# There is no current way to run an `or` operator in a filter, if you want a list of ASSIGNED incidents as well, you will need to change the filter and rerun.
response = requests.get(
    "http://api.gitguardian.com/v1/incidents/secrets",
    {"status":"TRIGGERED", "per_page":"10", "ordering":"-date"}, 
    headers=auth_headers 
    )
# {'page': 2} is deprecated but still works to get the next batch. 
# Better method would be to leverage `cursor` filter but that takes more logic.

# Let's store this JSON data in a var for ease of coding
incident_data = response.json()
print("Total incidents = " + str(len(incident_data)))

## okta work ####################
# Get the list of active users from Okta
response = requests.get(
    "https://{}.okta.com/api/v1/users?filter=status+eq+%22ACTIVE%22".format(okta_subdomain), 
    headers=okta_auth_header
    )

## Build the list of Active Okta users emails
active_user_email_list = []
for users in response.json():
    active_user_email_list.append(users['profile']['email'])
# print(active_user_email_list)

# Build a list of affected incidnets = where the developer is no longer active
affected_incidents = []
for incident in incident_data:
    this_incident_id = incident['id']
    response = requests.get(
        "https://api.gitguardian.com/v1/incidents/secrets/{}".format(this_incident_id), 
        headers=auth_headers
        )
    committer_email = response.json()['occurrences'][0]['author_info']
    if not any(committer_email in email for email in active_user_email_list):
        affected_incidents.append(this_incident_id)

print("Done")
print(affected_incidents)

## Okta 2 - Assign Affected Incidents To Approved Team Member

*The following code assumes you already ran the above code block and created a file called `affected_incidents`*
If you did not run the code, please do that now to proceed. 

Now that you have a list of the affected incidents, time to assign them to the correct member of the team.

In [ ]:
## Manually assign the email address of the relevant team member

# Unless you have Bruce on your team, make sure and update the next line
assignee_email="bruce-wayne-gg@protonmail.com"

# Let's make some reporting related variables. 
already_assigned_list = []
success_list = []
issue_list = []

print("Assigning "+ str(len(affected_incidents)) + " to " + assignee_email + " in 5 seconds")
#delay for 5 seconds - can kill if somethig looks off
time.sleep(5) 

for incident in affected_incidents:
    response = requests.post(
        "https://api.gitguardian.com/v1/incidents/secrets/{}/assign".format(incident), 
        headers=auth_headers, 
        data = {"email" : assignee_email}
        )
    if response.status_code == 409 :
        already_assigned_list.append(incident)
    elif response.status_code  == 200 :
        success_list.append(incident)
    else: 
        issue_list.append(incident)
        
print("Done")
print("Already assigned to this user = " + str(len(already_assigned_list))) 
print("Successfully assigned to this user = " + str(len(success_list)))
if len(issue_list) > 0:
    print("Hit some issues with assigning these =" + str(issue_list))
